In [1]:
from sklearn.preprocessing import StandardScaler,OneHotEncoder
import pandas as pd
from pathlib import Path
from collections import Counter
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import confusion_matrix
from imblearn.metrics import classification_report_imbalanced
import warnings
warnings.filterwarnings('ignore')

In [2]:
df_HR =pd.read_csv("Resources/hr.csv")
df_HR.head()

,enrollee_id,city,city_development_index,gender,relevent_experience,enrolled_university,education_level,major_discipline,experience,company_size,company_type,last_new_job,training_hours,target
0,8949,city_103,0.920,Male,Has relevent experience,no_enrollment,Graduate,STEM,20,50through99,Pvt Ltd,1,More,1.0
1,29725,city_40,0.776,Male,No relevent experience,no_enrollment,Graduate,STEM,15,50through99,Pvt Ltd,5,Between,0.0
2,11561,city_21,0.624,Other,No relevent experience,Full time course,Graduate,STEM,5,50through99,Pvt Ltd,0,Less,0.0
4,666,city_162,0.767,Male,Has relevent experience,no_enrollment,Masters,STEM,20,50through99,Funded Startup,4,More,0.0
5,21651,city_176,0.764,Other,Has relevent experience,Part time course,Graduate,STEM,11,50through99,Pvt Ltd,1,More,1.0


# Encoding

In [3]:
# Generate our categorical variable list
hr_cat = df_HR.dtypes[df_HR.dtypes == "object"].index.tolist()
hr_cat

['city',
 'gender',
 'relevent_experience',
 'enrolled_university',
 'education_level',
 'major_discipline',
 'company_size',
 'company_type',
 'training_hours']

In [4]:
# Check the number of unique values in each column
df_HR[hr_cat].nunique()

city                   123
gender                   3
relevent_experience      2
enrolled_university      3
education_level          5
major_discipline         6
company_size             8
company_type             6
training_hours           3
dtype: int64

In [5]:
# Create a OneHotEncoder instance
enc = OneHotEncoder(sparse=False)

# Fit and transform the OneHotEncoder using the categorical variable list
encode_df = pd.DataFrame(enc.fit_transform(df_HR[hr_cat]))

# Add the encoded variable names to the DataFrame
encode_df.columns = enc.get_feature_names_out(hr_cat)
encode_df.head()

,city_city_1,city_city_10,city_city_100,city_city_101,city_city_102,city_city_103,city_city_104,city_city_105,city_city_106,city_city_107,...,company_size_<10,company_type_Early Stage Startup,company_type_Funded Startup,company_type_NGO,company_type_Other,company_type_Public Sector,company_type_Pvt Ltd,training_hours_Between,training_hours_Less,training_hours_More
0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0


In [6]:
# Merge one-hot encoded features and drop the originals
df_HR = df_HR.merge(encode_df,left_index=True, right_index=True)
df_HR = df_HR.drop(hr_cat,1)
df_HR

,enrollee_id,city_development_index,experience,last_new_job,target,city_city_1,city_city_10,city_city_100,city_city_101,city_city_102,...,company_size_<10,company_type_Early Stage Startup,company_type_Funded Startup,company_type_NGO,company_type_Other,company_type_Public Sector,company_type_Pvt Ltd,training_hours_Between,training_hours_Less,training_hours_More
0,8949,0.920,20,1,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0
1,29725,0.776,15,5,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0
2,11561,0.624,5,0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
4,666,0.767,20,4,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0
5,21651,0.764,11,1,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18377,24169,0.897,14,5,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0
18378,26496,0.698,20,5,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0
18379,21543,0.887,9,1,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0
18380,30491,0.926,11,4,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0


# Split the Data into Training and Testing

In [7]:
# Create our features
x_cols = [i for i in df_HR.columns if i not in ('enrollee_id', 'target')]
X = df_HR[x_cols]
y = df_HR['target']

In [8]:
X.describe()

,city_development_index,experience,last_new_job,city_city_1,city_city_10,city_city_100,city_city_101,city_city_102,city_city_103,city_city_104,...,company_size_<10,company_type_Early Stage Startup,company_type_Funded Startup,company_type_NGO,company_type_Other,company_type_Public Sector,company_type_Pvt Ltd,training_hours_Between,training_hours_Less,training_hours_More
count,17641.000000,17641.000000,17641.000000,17641.000000,17641.000000,17641.000000,17641.000000,17641.000000,17641.000000,17641.000000,...,17641.000000,17641.000000,17641.000000,17641.000000,17641.000000,17641.000000,17641.000000,17641.000000,17641.000000,17641.000000
mean,0.831133,10.088997,2.015645,0.001417,0.004478,0.014172,0.003911,0.016212,0.232356,0.016212,...,0.067910,0.031291,0.053398,0.027379,0.006349,0.050224,0.831359,0.321070,0.304008,0.374922
std,0.122453,6.505694,1.675955,0.037619,0.066771,0.118201,0.062420,0.126294,0.422347,0.126294,...,0.251598,0.174107,0.224833,0.163191,0.079429,0.218413,0.374445,0.466901,0.459999,0.484117
min,0.448000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.743000,5.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000
50%,0.910000,9.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000
75%,0.920000,16.000000,3.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,1.000000,1.000000
max,0.949000,20.000000,5.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [9]:
# Check the balance of our target values
y.value_counts()

0.0    13324
1.0     4317
Name: target, dtype: int64

In [10]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y, random_state=78)
                                                   


# RandomOver Sampler ML

In [11]:
# classifier = LogisticRegression(solver='lbfgs', random_state=1)
# classifier

In [12]:
# # Train the data
# classifier.fit(X_train, y_train)

In [13]:
# Resample the training data with the RandomOversampler
from imblearn.over_sampling import RandomOverSampler
ros = RandomOverSampler(random_state=1)

X_resampled, y_resampled = ros.fit_resample(X_train, y_train)

Counter(y_resampled)


Counter({1.0: 10006, 0.0: 10006})

In [14]:
# Train the Logistic Regression model using the resampled data
# Logistic regression using random oversampled data
from sklearn.linear_model import LogisticRegression

model = LogisticRegression(solver='lbfgs', random_state=1)
model.fit(X_resampled, y_resampled)

LogisticRegression(random_state=1)

In [15]:
# Calculated the balanced accuracy score
from sklearn.metrics import balanced_accuracy_score
y_pred = model.predict(X_test)

balanced_accuracy_score(y_test, y_pred)

0.6537149110979122

In [16]:
# Display the confusion matrix
from sklearn.metrics import confusion_matrix

y_pred = model.predict(X_test)
confusion_matrix(y_test, y_pred)

array([[2547,  771],
       [ 503,  590]], dtype=int64)

In [17]:
# Print the imbalanced classification report
from imblearn.metrics import classification_report_imbalanced
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

        0.0       0.84      0.77      0.54      0.80      0.64      0.42      3318
        1.0       0.43      0.54      0.77      0.48      0.64      0.40      1093

avg / total       0.74      0.71      0.60      0.72      0.64      0.42      4411

